In [21]:
import numpy as np
import pandas as pd

In [12]:
def compute_API(lambda_max, n):
    return np.abs(lambda_max-n)/(n-1)

In [5]:
A = np.array([[1, 1.1038, 0.6888, 1.19, 7.8, 1.8235],
               [0.905, 1, 0.6241, 1.0955, 7.0801, 1.6403],
               [1.4501, 1.6026, 1, 1.7705, 11.32, 2.6506],
               [0.8893, 0.9099, 0.5677, 1, 6.43, 1.4957],
               [0.1282, 0.1414, 0.0883, 0.1555, 1, 0.2328],
               [0.5521, 0.6098, 0.3773, 0.6681, 4.3100, 1]])
A

array([[ 1.    ,  1.1038,  0.6888,  1.19  ,  7.8   ,  1.8235],
       [ 0.905 ,  1.    ,  0.6241,  1.0955,  7.0801,  1.6403],
       [ 1.4501,  1.6026,  1.    ,  1.7705, 11.32  ,  2.6506],
       [ 0.8893,  0.9099,  0.5677,  1.    ,  6.43  ,  1.4957],
       [ 0.1282,  0.1414,  0.0883,  0.1555,  1.    ,  0.2328],
       [ 0.5521,  0.6098,  0.3773,  0.6681,  4.31  ,  1.    ]])

In [117]:
%%time

eigvals, eigvects = np.linalg.eig(A)
idxmax = np.argmax(eigvals)
valmax = eigvals[idxmax]
valapi = compute_API(valmax, len(A))
vecmax = eigvects[:,idxmax]
print(idxmax, valmax, valapi)
print(vecmax)

if valapi>0:
    print('Arbitrage oportunity detected.')
else:
    print('Arbitrage oportunity no detected.')

0 (6.011554943772773+0j) 0.0023109887545546214
[-0.44852453+0.j -0.40701659+0.j -0.65381971+0.j -0.37545331+0.j
 -0.05763717+0.j -0.24778175+0.j]
Arbitrage oportunity detected.
CPU times: user 1.41 ms, sys: 0 ns, total: 1.41 ms
Wall time: 1.25 ms


In [131]:
%%time

B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
C = np.divide(A, B)

######
C_idx_max_i = int(C.argmax()/C.shape[0])
C_idx_max_j = C.argmax()%C.shape[0]
C_idx_min_i = int(C.argmin()/C.shape[0])
C_idx_min_j = C.argmin()%C.shape[0]

overvalued_idx = C_idx_max_i, C_idx_max_j
undervalued_idx = C_idx_min_i, C_idx_min_j
overvalued_val = C[overvalued_idx]
undervalued_val = C[undervalued_idx]

bridge_idx = C_idx_min_j, C_idx_max_j
base_currency_idx = C_idx_min_j, C_idx_min_j
bridge_val = C[bridge_idx]

######
print('Arbitrage orders:')
print(f'BUY  foreign currency EUR/YEN in Frankfurt(EUR):  1 / {A[C_idx_max_i,C_idx_min_j]:.4} = x YEN')
print(f'SELL foreign currency YEN/USDT in New York(USDT): x * {A[C_idx_max_i, C_idx_max_j]:.4} = y USDT')
print(f'BUY  foreign currency UDST/EUR in New York(USDT): y / {A[C_idx_min_j, C_idx_max_j]:.4} = z EUR')
z = 1/A[C_idx_max_i,C_idx_min_j] * A[C_idx_max_i, C_idx_max_j] / A[C_idx_min_j, C_idx_max_j]
print(f'There is a increase of {z-1:.3%}')

Arbitrage orders:
BUY  foreign currency EUR/YEN in Frankfurt(EUR):  1 / 0.9099 = x YEN
SELL foreign currency YEN/USDT in New York(USDT): x * 0.8893 = y USDT
BUY  foreign currency UDST/EUR in New York(USDT): y / 0.905 = z EUR
There is a increase of 7.996%
CPU times: user 1.3 ms, sys: 197 µs, total: 1.5 ms
Wall time: 1.23 ms


In [119]:
undervalued_val

(0.9863926872898054+0j)

In [61]:
(C_idx_max_i+1, C_idx_max_j+1), (C_idx_min_i+1, C_idx_min_j+1)

((4, 1), (4, 2))